In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 7.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.request import urlopen
import json
import time
import pandas as pd
import io

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotVisibleException




In [2]:
dining_list = pd.read_csv("dining_listup_kakao3.csv")

In [5]:
new_dining_list = dining_list

In [136]:
len(new_dining_list)

477

In [164]:
new_dining_list

,dining_id,total_name,dining_name,branch_name,category,addr,source
0,0,카덴 연희본점,카덴,연희본점,일식,서울 서대문구 연희로 173 거화빌딩 1층,카카오
1,1,앤트러사이트 연희점,앤트러사이트,연희점,커피전문점,서울 서대문구 연희로 135,카카오
2,2,목란,목란,NaN,중식,서울 서대문구 연희로15길 21,카카오
3,3,콘하스 연희점,콘하스,연희점,카페,서울 서대문구 연희로27길 99,카카오
4,4,연경 연희점,연경,연희점,중화요리,서울 서대문구 연희로 145,카카오
...,...,...,...,...,...,...,...
472,473,히우카우,히우카우,NaN,태국음식,서울 서대문구 연세로4길 61 1층,카카오
473,474,연남서식당,연남서식당,NaN,갈비,서울 마포구 백범로2길 32,카카오
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",서울 서대문구 명물길 27-18 1층,카카오
475,476,포티드,포티드,NaN,카페,서울 서대문구 명물길 36-3 1층,카카오


In [6]:
new_dining_list['addr'] = dining_list.addr.map(lambda x : x.split("\n")[1])

In [7]:
new_dining_list

,dining_id,total_name,dining_name,branch_name,category,addr,source
0,0,카덴 연희본점,카덴,연희본점,일식,서울 서대문구 연희로 173 거화빌딩 1층,카카오
1,1,앤트러사이트 연희점,앤트러사이트,연희점,커피전문점,서울 서대문구 연희로 135,카카오
2,2,목란,목란,NaN,중식,서울 서대문구 연희로15길 21,카카오
3,3,콘하스 연희점,콘하스,연희점,카페,서울 서대문구 연희로27길 99,카카오
4,4,연경 연희점,연경,연희점,중화요리,서울 서대문구 연희로 145,카카오
...,...,...,...,...,...,...,...
472,473,히우카우,히우카우,NaN,태국음식,서울 서대문구 연세로4길 61 1층,카카오
473,474,연남서식당,연남서식당,NaN,갈비,서울 마포구 백범로2길 32,카카오
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",서울 서대문구 명물길 27-18 1층,카카오
475,476,포티드,포티드,NaN,카페,서울 서대문구 명물길 36-3 1층,카카오


In [385]:
#웹드라이버 로딩
driver = webdriver.Chrome('/Users/pdh/TasteMaps/TasteMap/crawler/chromedriver')
driver.implicitly_wait(3)

In [217]:
naver_dining_id_list = []
count_check_list = []
no_exist_check_list = []

In [218]:

naver_url = "https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query="

for i in range(len(new_dining_list)) :
    addr = new_dining_list.iloc[i]['addr']
    dining_name = new_dining_list.iloc[i]['dining_name']
    query = addr + " " + dining_name
    driver.get(naver_url + query)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    candidates = soup.find_all(class_ = '_3g_0T')
    naver_dining_id = ""
    count = 0

    for c in candidates :
        c_name = c.text
        end_index = c.attrs['href'].find("?c")
        c_id = c.attrs['href'][37:end_index]
        print(c_name)
        print(c_id)
        if c_name == dining_name :
            naver_dining_id = c_id
            count = 1
            break;

        if dining_name in c_name or c_name in dining_name:
            print("here")
            count += 1
            naver_dining_id = c_id

    #naver_dining_id = int(naver_dining_id)

    if count > 1:
        print("########Check : ", naver_url + query, "###########")
        count_check_list.append(naver_url + query)
        
    if not naver_dining_id:
        print("########Check : ", naver_url + query, "###########")
        no_exist_check_list.append(naver_url + query)
        
    naver_dining_id_list.append(naver_dining_id)

우동카덴
380995011
here
카덴
218413355
앤트러사이트 연희점
1485444554
here
목란
11880458
콘하스
97642371
연경 연희점
1721729164
here
피터팬1978
13413865
수빈
11679474
연희동칼국수 본점
11727641
here
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로26길 24 고미정 ###########
재인
1039110015
매뉴팩트커피
32982001
라이라이 본점
1588882724
here
알파룸
1105318015
젤로데이
1295766562
시오
37687821
로덴드론 연희점
1595735508
here
녹원쌈밥
19899218
조은집
11832249
폴앤폴리나
709038120
스웨이커피스테이션
1826959498
작은스페인
38738511
매뉴팩트커피
32982001
에브리띵베이글
1588980015
연희김밥 본점
18752158
here
스쿨푸드 연희 딜리버리점
272273213
쥬씨 연희점
38473771
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 증가로 17 연희곰탕 ###########
이화원
11555246
책바
37069823
르솔레이
1915794004
효동각본점
37792280
here
SICTAC 하늬솔점
37585958
함평면옥
11679712
몸뚱아리 운동센터
38264833
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 성산로 527 하늬솔빌딩 1층 식탁

미스터서왕만두
36670196
거북이집
33021498
부추곱창
1865807926
명륜진사갈비 서울신촌점
1491408912
here
서울의집
1319788247
에이지케이코리아
1927616082
진보
11555291
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 성산로 531 아키 ###########
쁠라뒤쥬르
36899969
알레스구떼
1320970305
타코벨 신촌점
38676295
here
제오헤어 신촌점
38642772
로파우사다 신촌점
13413910
육회이야기 연희점
1722361045
here
초가집삼겹살좋아
1314052927
케이엠어학원
21874752
포석정
11603484
밀스
37554274
엠드엠
1636887683
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로11가길 53 페퍼민트패티 ###########
뱅센느
13119742
하꾸야
1436862783
오마
1404784472
유자유김치떡볶이 신촌점
1197641297
here
119등갈비
1867051094
사라바 쉐어하우스 1호점
1583674689
북문아구찜
18257823
커피리브레 연남점
21856573
오복수산
37728675
몽중식
1476690926
고래식당 연희점
1459564350
here
와이즈만 서대문상암센터
13116392
엘스스튜디오
1378742572
연희단팥죽
1124058036
연희옥 연희점
1465790494
here
쏘렌토 연희점
1225771794
에프터샵
1766886261
뉴욕비앤씨 신촌점
34761579
here
뉴욕비 스카치에그&비스트로 삼겹도시락
1827935587
오랑지
1771873244
배바위

방탈출 룸익스케이프 화이트점
689304082
꾼낚시카페 홍대신촌점
37762110
here
문베어
1437237006
리리코
1841187535
스시올리지
1038842069
행복감
1490934703
월순철판동태찜
13329483
레이씨
1861292357
소담식당
1623755884
가야가야
13507856
쉐프 수
37932710
코코앤영
1758591246
낭만주객
1374800693
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희맛로 7-8 1층 낭만치맥 ###########
압구정샤브샤브 신촌점
11560770
here
L5 The Restaurant
847547218
CU 연세동문점
35743565
오디너리핏
1934994866
원앤온리커피
1507115682
오렌지룸 신촌점
18244176
놀숲 신촌점
37391652
착한돼지 신촌점
32397728
here
스탬프커피
1859539507
상해소흘
1251267977
홍대개미 신촌점
880225042
here
포토페이스
36112263
종로유학원 신촌지사
35014173
하루돈까스 신촌본점
11713759
here
정연빌딩
20735795
롯지190
1604858931
후지산
18248963
바프로
31375533
아우프글렛 연남점
1565283628
here
03 인앤아웃
38779519
브루멜테일러
1059809414
jason95
1729872020
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로11길 57 03인앤아웃 ###########
########Check :  https://search.naver.com/search.naver?where=nex

제임스시카고피자 신촌점
36877608
here
315 bar
1918512095
치킨버스 신촌본점
1306481836
연남화로
1051445603
오복수산시장
674179089
더라스트오더
1240256818
경양카츠 연남점
1128364627
here
두껍다숙성뒷고기
1414142434
오비베어 연남점
1291233476
리소헤어 연세로점
37008926
메종드시크릿
49248974
here
쥬씨 신촌점
1765188454
지구별고양이
36528222
타코야 연희점
318771898
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로12길 10-10 101호 타코야치킨하우스 ###########
오아시스 연희점
1322024369
왕언니갈비탕
1293512571
올랑올랑
1179975511
쿄 라멘
1636571197
한가커피
1696325971
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 마포구 동교로46길 25 지층 쿄라멘 ###########
삼미식당 신촌점
1778646844
here
연남동질리
1733564552
here
오직고기 연남점
1386439657
신군신양
38010159
WP커피
1388383414
와이제이빌
20897959
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로 89-11 1층 더블유피 ###########
수라간
1063954725
김덕후의차돌조 신촌점
519257167
here
나비어
32448391
작은나폴리
35092955
명랑핫

In [219]:
len(naver_dining_id_list)

477

In [137]:
naver_dining_id_list

['218413355',
 '1485444554',
 '11880458',
 '97642371',
 '1721729164',
 '13413865',
 '11679474',
 '11727641',
 '',
 '1039110015',
 '218413355',
 '1485444554',
 '11880458',
 '97642371',
 '1721729164',
 '13413865',
 '11679474',
 '11727641',
 '',
 '',
 '32982001',
 '1588882724',
 '37687821',
 '1595735508',
 '19899218',
 '',
 '709038120',
 '1826959498',
 '38738511',
 '1588980015',
 '18752158',
 '1345504483',
 '11555246',
 '37069823',
 '1915794004',
 '37792280',
 '',
 '37049500',
 '34326631',
 '13329483',
 '38003033',
 '19877438',
 '33704447',
 '1287085178',
 '',
 '1019654977',
 '37101657',
 '1292205602',
 '36543730',
 '18248947',
 '37910087',
 '1038526987',
 '',
 '32875867',
 '1288247568',
 '37039747',
 '1902737844',
 '628641878',
 '35092955',
 '1904793015',
 '36943314',
 '35563009',
 '1526042047',
 '35624223',
 '',
 '31608233',
 '13150869',
 '31009691',
 '1702893591',
 '',
 '',
 '1685805793',
 '38286911',
 '37663300',
 '1852952548',
 '',
 '11849155',
 '37648608',
 '864066117',
 '1437366482

In [221]:
len(no_exist_check_list)

125

In [220]:
len(count_check_list)

0

In [222]:
id_df = pd.DataFrame(naver_dining_id_list)

In [223]:
id_df = id_df.rename({0:'naver_dining_id'},axis=1)

In [224]:
new_df = pd.concat([new_dining_list,id_df],axis=1, join='inner')

In [225]:
new_df

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
0,0,카덴 연희본점,카덴,연희본점,일식,서울 서대문구 연희로 173 거화빌딩 1층,카카오,218413355
1,1,앤트러사이트 연희점,앤트러사이트,연희점,커피전문점,서울 서대문구 연희로 135,카카오,1485444554
2,2,목란,목란,NaN,중식,서울 서대문구 연희로15길 21,카카오,11880458
3,3,콘하스 연희점,콘하스,연희점,카페,서울 서대문구 연희로27길 99,카카오,97642371
4,4,연경 연희점,연경,연희점,중화요리,서울 서대문구 연희로 145,카카오,1721729164
...,...,...,...,...,...,...,...,...
472,473,히우카우,히우카우,NaN,태국음식,서울 서대문구 연세로4길 61 1층,카카오,
473,474,연남서식당,연남서식당,NaN,갈비,서울 마포구 백범로2길 32,카카오,11708756
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",서울 서대문구 명물길 27-18 1층,카카오,
475,476,포티드,포티드,NaN,카페,서울 서대문구 명물길 36-3 1층,카카오,1946991741


In [230]:
no_exist_check_list[:10]

['https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로26길 24 고미정',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 증가로 17 연희곰탕',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 성산로 527 하늬솔빌딩 1층 식탁',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로11길 20 이품',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희맛로 6 청송함흥냉면전문점 본관',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로14길 5 호천식당',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로 32 만동제과',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 증가로 13-9 2층 작은나폴리',
 'https://search.naver.com/se

In [208]:
new_df.loc[new_df['dining_name'] == '일룸 엄마의서재',['naver_dining_id']] = '1816938991'

In [231]:
new_df.loc[new_df['dining_name'] == '연희곰탕']

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
21,21,연희곰탕,연희곰탕,NaN,곰탕,서울 서대문구 증가로 17,카카오,


In [232]:
no_dining_id = new_df[new_df.naver_dining_id == ""]

In [378]:
len(no_dining_id)

79

In [386]:
naver_dining_id_list_2 = []
count_check_list_2 = []
no_exist_check_list_2 = []

In [387]:

naver_url = "https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query="

for i in range(len(no_dining_id)) :
    addr = no_dining_id.iloc[i]['addr']
    dining_name = no_dining_id.iloc[i]['dining_name']
    query = addr + " " + dining_name
    driver.get(naver_url + query)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    candidates = soup.find_all(class_ = '_3g_0T')
    naver_dining_id = ""
    count = 0

    for c in candidates :
        c_name = c.text
        end_index = c.attrs['href'].find("?c")
        c_id = c.attrs['href'][37:end_index]
        print(c_name)
        print(c_id)
        if c_name == dining_name :
            naver_dining_id = c_id
            count = 1
            break;

        if dining_name in c_name or c_name in dining_name:
            print("here")
            count += 1
            naver_dining_id = c_id
            break;

    #naver_dining_id = int(naver_dining_id)

    if count > 1:
        print("########Check : ", naver_url + query, "###########")
        count_check_list_2.append(naver_url + query)
        
    if not naver_dining_id:
        print("########Check : ", naver_url + query, "###########")
        no_exist_check_list_2.append(naver_url + query)
        
    naver_dining_id_list_2.append(naver_dining_id)

########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로26길 24 고미정 ###########
SICTAC 하늬솔점
37585958
함평면옥
11679712
몸뚱아리 운동센터
38264833
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 성산로 527 하늬솔빌딩 1층 식탁 ###########
청송본관함흥냉면전문점
11725959
기리보이월드
1891640639
청송면옥
1207981597
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희맛로 6 청송함흥냉면전문점 본관 ###########
유어마인드
153796899
초콜릿코스모스
33373166
비하인드 리메인
71969112
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로11라길 10-6 비하인드리메인 ###########
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 마포구 성미산로 200 커피리브레 ###########
엠브로 돈까스
1815955748
니뽕내뽕 신촌점
30808658
로빈스에그 만화카페
37630199
########Check :  https://search.naver.c

신촌세브란스병원
11693113
신촌세브란스병원연세암병원
21198405
신촌세브란스병원어린이병원
38693373
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연세로 50-1 지하1층 파리크라상 ###########
신촌칼
262362736
신촌아지트
1166296669
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연세로4길 42-3 신촌 칼 ###########
아담앤이브왁싱&태닝
35652474
방탈출 제로
343426775
정가
37992818
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 명물길 49 5층 방탈출제로 ###########
고래식당 연희점
1459564350
와이즈만 서대문상암센터
13116392
엘스스튜디오
1378742572
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희맛로 32 1층 101호 도시수산시장 ###########
HONNE
523525227
그리너리 테이블
999445330
댄싱케이크
1779659383
########Check :  https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 마포구 동교로46길 42-3 1층 혼네 #########

In [388]:
len(naver_dining_id_list_2)

79

In [391]:
len(no_exist_check_list_2)

79

In [390]:
len(count_check_list_2)

0

In [247]:
temp = new_df

In [248]:
temp

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
0,0,카덴 연희본점,카덴,연희본점,일식,서울 서대문구 연희로 173 거화빌딩 1층,카카오,218413355
1,1,앤트러사이트 연희점,앤트러사이트,연희점,커피전문점,서울 서대문구 연희로 135,카카오,1485444554
2,2,목란,목란,NaN,중식,서울 서대문구 연희로15길 21,카카오,11880458
3,3,콘하스 연희점,콘하스,연희점,카페,서울 서대문구 연희로27길 99,카카오,97642371
4,4,연경 연희점,연경,연희점,중화요리,서울 서대문구 연희로 145,카카오,1721729164
...,...,...,...,...,...,...,...,...
472,473,히우카우,히우카우,NaN,태국음식,서울 서대문구 연세로4길 61 1층,카카오,
473,474,연남서식당,연남서식당,NaN,갈비,서울 마포구 백범로2길 32,카카오,11708756
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",서울 서대문구 명물길 27-18 1층,카카오,
475,476,포티드,포티드,NaN,카페,서울 서대문구 명물길 36-3 1층,카카오,1946991741


In [250]:
check_df = pd.DataFrame(naver_dining_id_list_2)

In [307]:
check_df

,0
8,
21,1345504483
26,
31,19877438
34,
...,...
460,34079145
461,
463,1560785276
472,1758308742


In [308]:
new_df = pd.concat([no_dining_id,check_df],axis=1)

In [309]:
new_df = new_df.drop(["naver_dining_id"],axis=1)

In [262]:
check_df.index = no_dining_id.index

In [310]:
new_df = new_df.rename({0:"naver_dining_id"},axis=1)

In [315]:
new_df

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
8,8,고미정,고미정,NaN,한정식,서울 서대문구 연희로26길 24,카카오,
21,21,연희곰탕,연희곰탕,NaN,곰탕,서울 서대문구 증가로 17,카카오,1345504483
26,26,식탁 하늬솔점,식탁,하늬솔점,이탈리안,서울 서대문구 성산로 527 하늬솔빌딩 1층,카카오,
31,31,이품,이품,NaN,중화요리,서울 서대문구 연희로11길 20,카카오,19877438
34,34,청송함흥냉면전문점 본관,청송함흥냉면전문점 본관,NaN,냉면,서울 서대문구 연희맛로 6,카카오,
...,...,...,...,...,...,...,...,...
460,461,데일리라운드,데일리라운드,NaN,카페,서울 마포구 동교로38길 27-19 2층,카카오,34079145
461,462,회장님댁 신촌점,회장님댁,신촌점,"호프,요리주점",서울 서대문구 연세로7안길 21 1층,카카오,
463,464,희로,희로,NaN,일본식주점,서울 서대문구 성산로 325,카카오,1560785276
472,473,히우카우,히우카우,NaN,태국음식,서울 서대문구 연세로4길 61 1층,카카오,1758308742


In [276]:
temp2 = temp

In [305]:
temp

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
0,0,카덴 연희본점,카덴,연희본점,일식,서울 서대문구 연희로 173 거화빌딩 1층,카카오,218413355
1,1,앤트러사이트 연희점,앤트러사이트,연희점,커피전문점,서울 서대문구 연희로 135,카카오,1485444554
2,2,목란,목란,NaN,중식,서울 서대문구 연희로15길 21,카카오,11880458
3,3,콘하스 연희점,콘하스,연희점,카페,서울 서대문구 연희로27길 99,카카오,97642371
4,4,연경 연희점,연경,연희점,중화요리,서울 서대문구 연희로 145,카카오,1721729164
...,...,...,...,...,...,...,...,...
472,473,히우카우,히우카우,NaN,태국음식,서울 서대문구 연세로4길 61 1층,카카오,
473,474,연남서식당,연남서식당,NaN,갈비,서울 마포구 백범로2길 32,카카오,11708756
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",서울 서대문구 명물길 27-18 1층,카카오,
475,476,포티드,포티드,NaN,카페,서울 서대문구 명물길 36-3 1층,카카오,1946991741


In [321]:
index_list = temp[temp['naver_dining_id'] == ""].index


In [357]:
temp2.loc[8,'naver_dining_id']

''

In [361]:
temp2.loc[8,'naver_dining_id'] = ""

In [362]:
temp2.iloc[8]

dining_id                          8
total_name                       고미정
dining_name                      고미정
branch_name                      NaN
category                         한정식
addr               서울 서대문구 연희로26길 24
source                           카카오
naver_dining_id                     
Name: 8, dtype: object

In [341]:
temp2.iloc[8]['naver_dining_id']

''

In [363]:
for index in index_list :
    for i in range(len(new_df)) :
        if temp2.iloc[index]['dining_id'] == new_df.iloc[i]['dining_id'] :
            print("here:",index)
            temp2.loc[index,'naver_dining_id'] = new_df.iloc[i]['naver_dining_id']
            break;


here: 8
here: 21
here: 26
here: 31
here: 34
here: 40
here: 41
here: 48
here: 54
here: 59
here: 65
here: 73
here: 75
here: 83
here: 87
here: 89
here: 96
here: 97
here: 98
here: 101
here: 103
here: 109
here: 110
here: 115
here: 117
here: 119
here: 120
here: 122
here: 128
here: 129
here: 130
here: 133
here: 149
here: 157
here: 172
here: 179
here: 184
here: 189
here: 194
here: 198
here: 200
here: 201
here: 204
here: 206
here: 208
here: 215
here: 216
here: 222
here: 223
here: 224
here: 225
here: 232
here: 235
here: 236
here: 239
here: 242
here: 249
here: 253
here: 260
here: 261
here: 266
here: 267
here: 275
here: 288
here: 289
here: 292
here: 295
here: 297
here: 298
here: 303
here: 307
here: 310
here: 316
here: 318
here: 326
here: 328
here: 332
here: 334
here: 336
here: 337
here: 338
here: 339
here: 340
here: 341
here: 343
here: 352
here: 353
here: 356
here: 359
here: 360
here: 364
here: 365
here: 368
here: 370
here: 372
here: 373
here: 384
here: 386
here: 389
here: 392
here: 394
here: 396


In [369]:
temp = temp2

In [373]:
no_dining_id = temp[temp['naver_dining_id'] == ""]

In [374]:
no_dining_id

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
8,8,고미정,고미정,NaN,한정식,서울 서대문구 연희로26길 24,카카오,
26,26,식탁 하늬솔점,식탁,하늬솔점,이탈리안,서울 서대문구 성산로 527 하늬솔빌딩 1층,카카오,
34,34,청송함흥냉면전문점 본관,청송함흥냉면전문점 본관,NaN,냉면,서울 서대문구 연희맛로 6,카카오,
54,54,비하인드리메인,비하인드리메인,NaN,카페,서울 서대문구 연희로11라길 10-6,카카오,
59,59,커피리브레 연남점,커피리브레,연남점,커피전문점,서울 마포구 성미산로 200,카카오,
...,...,...,...,...,...,...,...,...
439,440,코블러연희,코블러연희,NaN,칵테일바,서울 서대문구 연희맛로 31-9 1층,카카오,
453,454,와플대학 연희캠퍼스,와플대학 연희캠퍼스,NaN,"제과,베이커리",서울 서대문구 연희로 90 1층,카카오,
459,460,두끼떡볶이 이대점,두끼떡볶이,이대점,떡볶이,서울 서대문구 이화여대길 89 2층,카카오,
461,462,회장님댁 신촌점,회장님댁,신촌점,"호프,요리주점",서울 서대문구 연세로7안길 21 1층,카카오,


In [521]:
no_exist_check_list_2[70:]

['https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연세로5길 4 1층 타이거슈가',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연세로9길 14 칠칠켄터키',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로11길 7 지하1층 크래프트비어칸',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로 153 1층 나나하치',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희맛로 31-9 1층 코블러연희',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연희로 90 1층 와플대학 연희캠퍼스',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 이화여대길 89 2층 두끼떡볶이',
 'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=1&acr=1&ie=utf8&query=서울 서대문구 연세로7안길 21 1층 회장님댁',
 'http

In [529]:
ddf.loc[ddf['dining_name']=='비아메렝게']

,dining_id,total_name,dining_name,branch_name,category,addr,source,naver_dining_id
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",서울 서대문구 명물길 27-18 1층,카카오,225603655


In [528]:
ddf.loc[ddf['dining_name']=='비아메렝게','naver_dining_id'] = '225603655'

In [534]:
ddf.to_csv("dining_listup_with_naver_id.csv", index=False, encoding = "utf-8-sig")